In [1]:
import torch
import pandas as pd
from torch import nn
import pytorch_lightning as pl
import torch.utils.data as data_utils
import numpy as np
from os import cpu_count
from types import NoneType

/home/kacper/anaconda3/envs/gpu_torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import wandb
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint, LearningRateFinder
from pytorch_lightning.loggers import WandbLogger

In [3]:
torch.set_float32_matmul_precision("medium")

In [4]:
df=pd.read_parquet('./data/books.par')
df.columns

Index(['title', 'genre', 'summary', 'input_ids', 'att_mask', 'label',
       'mapped_inputs'],
      dtype='object')

# dataset

In [5]:
class BookDataset(data_utils.Dataset):
    def __init__(self, df: pd.DataFrame):
        self.input_ids=torch.tensor(df.mapped_inputs, dtype=torch.int32)
        self.att_masks=torch.tensor(df.att_mask, dtype=torch.float32)
        self.y=torch.tensor(df.label.values, dtype=torch.uint8)
        self.no_classes=df.label.nunique()

    def __len__(self):
        return self.y.shape[0]
    
    def __getitem__(self, indexes):
        return self.input_ids[indexes], self.att_masks[indexes], self.y[indexes]
        

In [6]:
df=df.iloc[np.random.permutation(df.shape[0])].reset_index(drop=True)
split=int(df.shape[0]*0.9)

train_df=df.iloc[:split]
val_df=df.iloc[split:].reset_index(drop=True)

In [7]:
_,label_weights=np.unique(train_df.label, return_counts=True)
label_weights=1/label_weights
label_weights=label_weights/np.sum(label_weights)
label_weights

array([0.02343692, 0.03102526, 0.0411588 , 0.03394462, 0.0338821 ,
       0.01999781, 0.21147106, 0.19164564, 0.2067189 , 0.2067189 ])

In [8]:
train_data=BookDataset(train_df)
val_data=BookDataset(val_df)

train_dataloader=data_utils.DataLoader(train_data, batch_size=32, num_workers=cpu_count(),
                                       shuffle=True, drop_last=True)
val_dataloader=data_utils.DataLoader(val_data, batch_size=32, num_workers=cpu_count())

/tmp/ipykernel_79012/3822734108.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  self.input_ids=torch.tensor(df.mapped_inputs, dtype=torch.int32)


# pl module

In [9]:
class BookGenreClassifier(pl.LightningModule):
    def __init__(self, model, lr=1e-2, loss=nn.CrossEntropyLoss(), l2=1e-5, lr_dc_step=3, lr_dc=0.1, **kwargs):
        super().__init__()
        self.save_hyperparameters(ignore=['model','loss'])
        if isinstance(loss.weight, NoneType):
            weighted_loss=False
        else: weighted_loss=True
        self.save_hyperparameters({'name':model.name, 
                                   'dropout_p':model.dropout_p,
                                   'weighted_loss':weighted_loss})
        self.lr=lr
        self.loss=loss
        self.model=model

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x = batch[:-1]
        y = batch[-1]
        logits=self(x)
        loss=self.loss(logits, y)

        self.log('train_loss', loss, prog_bar=True)
        return loss
    
    def evaluate(self, batch, mode=None):
        x = batch[:-1]
        y = batch[-1]
        logits=self(x)

        loss=self.loss(logits, y)

        preds=torch.argmax(logits, axis=1)
        acc=torch.sum(preds==y)/y.shape[0]
        # TODO add more metrics

        if mode:
            self.log(mode+'_loss', loss,  prog_bar=True)
            self.log(mode+'_acc', 100*acc,  prog_bar=True)

    def validation_step(self, batch, *args, **kwargs):
        return self.evaluate(batch, "val")
    def test_step(self, batch, *args, **kwargs):
        return self.evaluate(batch, "test")
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.model.parameters(), lr=self.lr, weight_decay=self.hparams.l2
        )
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            patience=self.hparams.lr_dc_step,
            factor=self.hparams.lr_dc,
            cooldown=1,
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss",
                "strict": False,
                "interval": "epoch",
                "frequency": 1,
                "name": "scheduler_lr",
            },
        }

# different models

## dummy example

In [128]:
class DummyModel(nn.Module): 
    def __init__(self, in_dim=7031, hid_dim=128, out_dim=10):
        # dummy model as an example, just one hidden layer straight from list of tokens
        super().__init__()
        self.name='DummyModel'
        self.l1=nn.Linear(in_dim, hid_dim)
        self.nonlinear=nn.Tanh()
        self.l2=nn.Linear(hid_dim, out_dim)

    def forward(self, x):
        in_ids, att_mask = x
        x=in_ids*att_mask
        x=self.l1(x)
        x=self.nonlinear(x)
        return self.l2(x)

### train

In [169]:
dm_model=BookGenreClassifier(DummyModel(), loss=nn.CrossEntropyLoss(weight=torch.tensor(label_weights, dtype=torch.float32)))

In [167]:
trainer=pl.Trainer(max_epochs=10)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [170]:
trainer.fit(dm_model, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | loss  | CrossEntropyLoss | 0     
1 | model | DummyModel       | 901 K 
-------------------------------------------
901 K     Trainable params
0         Non-trainable params
901 K     Total params
3.606     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 130/130 [00:02<00:00, 61.23it/s, v_num=2, train_loss=2.300, val_loss=2.310, val_acc=12.90] 

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 130/130 [00:02<00:00, 60.53it/s, v_num=2, train_loss=2.300, val_loss=2.310, val_acc=12.90]


# global variables

In [10]:
train_data[0]

(tensor([  302,  4950,   302,  ..., 36332, 36332, 36332], dtype=torch.int32),
 tensor([1., 1., 1.,  ..., 0., 0., 0.]),
 tensor(4, dtype=torch.uint8))

In [11]:
tokens_set=set()
for tokens in df.mapped_inputs.values:
    tokens_set=tokens_set.union(set(tokens))
vocab_size=len(tokens_set)
vocab_size

36333

# w2v

In [ ]:
class Word2VecModel(nn.Module):
    def __init__(self, hid_dim, out_dim, vocab_size):
        super().__init__()
        self.name='Word2VecModel'
        self.dropout_p=0

       # load embedigns from pretrained word2vec
        self.emb=nn.Embedding()#TODO
        self.out_layer=nn.Linear(hid_dim, out_dim)

    def forward(self, x):
        inputs, _=x
        w2v_output=self.emb(inputs)
        avg_output=# TODO śtednia
        return self.out_layer(avg_output)

# RNN

In [206]:
class SimpleRNN(nn.Module):
    def __init__(self, hid_dim, out_dim, vocab_size, dropout_p=0):
        super().__init__()
        self.name='SimpleRNN'
        self.dropout_p=dropout_p

        # last token states 'end of string' and is repeated multiple time at the end of an input
        # therefore set its embedding to 0 with padding_idx=-1
        self.emb=nn.Embedding(vocab_size, hid_dim, padding_idx=-1) 
        self.model=nn.RNN(input_size=hid_dim, hidden_size=hid_dim, batch_first=True, dropout=dropout_p)
        self.out_layer=nn.Linear(hid_dim, out_dim)

    def forward(self, x):
        inputs, _=x
        rnn_input=self.emb(inputs)
        _, hn=self.model(rnn_input)
        #use last hidden state as sequence representation
        hn=hn.squeeze()
        return self.out_layer(hn)

torch.Size([32, 7031])

In [231]:
rnn_model=BookGenreClassifier(SimpleRNN(128, 10, vocab_size=vocab_size), 
                             loss=nn.CrossEntropyLoss(),#weight=torch.tensor(label_weights, dtype=torch.float32),
                             lr_dc=0.5,
                             lr_dc_step=4,
                             )

In [233]:
wandb_logger = WandbLogger(
        project="ecoNLP", entity="kpuchalskixiv", log_model=True
    )

In [234]:
trainer=pl.Trainer(max_epochs=50,
                   callbacks=[
            EarlyStopping(
                monitor="val_acc", patience=25, mode="max", check_finite=True, check_on_train_epoch_end=False
            ),
            LearningRateMonitor(),
            ModelCheckpoint(monitor="val_acc", mode="max"),
         #   LearningRateFinder(min_lr=1e-4, num_training_steps=1000)
            ],
            logger=wandb_logger,
        )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [235]:
trainer.fit(rnn_model, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | loss  | CrossEntropyLoss | 0     
1 | model | SimpleRNN        | 4.7 M 
-------------------------------------------
4.7 M     Trainable params
0         Non-trainable params
4.7 M     Total params
18.740    Total estimated model params size (MB)


Epoch 49: 100%|██████████| 130/130 [00:05<00:00, 23.50it/s, v_num=xeoj, train_loss=2.140, val_loss=2.010, val_acc=23.00]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 130/130 [00:05<00:00, 23.49it/s, v_num=xeoj, train_loss=2.140, val_loss=2.010, val_acc=23.00]


In [236]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
scheduler_lr,██████████▄▄▄▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train_loss,▆▅▄▄▃▃▂▄█▄▃▅▄▂▃▃▂▂▃▂▃▃▂▂▃▂▂▂▃▃▂▂▂▄▂▁▃▂▁▄
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▁▃▆▅▆▆▃▅▅▆▅▅▆▅▆▆▇▄▇▇▆▆█▆▅▇▇██▆▅█▇▇▇▇▇▇▆▇
val_loss,▇▅▄▇▄▃▆█▅▄▄▄▃▃▂▂▂▃▃▂▁▂▂▂▃▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁
epoch,49
scheduler_lr,0.00031
train_loss,2.14065
trainer/global_step,6499
val_acc,22.96137


# LSTM

In [12]:
class SimpleLSTM(nn.Module):
    def __init__(self, hid_dim, out_dim, vocab_size, dropout_p=0):
        super().__init__()
        self.name='SimpleLSTM'
        self.dropout_p=dropout_p
        # last token states 'end of string' and is repeated multiple time at the end of an input
        # therefore set its embedding to 0 with padding_idx=-1
        self.emb=nn.Embedding(vocab_size, hid_dim, padding_idx=-1) 
        self.model=nn.LSTM(input_size=hid_dim, hidden_size=hid_dim, batch_first=True, dropout=dropout_p)
        self.out_layer=nn.Linear(hid_dim, out_dim)

    def forward(self, x):
        inputs, _=x
        rnn_input=self.emb(inputs)

        h0 = torch.randn(1, rnn_input.shape[0], rnn_input.shape[-1], device=rnn_input.device)
        c0 = torch.randn(1, rnn_input.shape[0], rnn_input.shape[-1], device=rnn_input.device)
        _, (hn, _)=self.model(rnn_input, (h0, c0))
        #use last hidden state as sequence representation
        hn=hn.squeeze()
        return self.out_layer(hn)

In [13]:
lstm_model=BookGenreClassifier(SimpleLSTM(128, 10, vocab_size=vocab_size, dropout_p=0.2137), 
                             loss=nn.CrossEntropyLoss(),#weight=torch.tensor(label_weights, dtype=torch.float32),
                             lr_dc=0.5,
                             lr_dc_step=4,
                             )

/home/kacper/anaconda3/envs/gpu_torch/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2137 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [14]:
wandb_logger = WandbLogger(
        project="ecoNLP", entity="kpuchalskixiv", log_model=True
    )

In [15]:
trainer=pl.Trainer(max_epochs=50,
                   callbacks=[
            EarlyStopping(
                monitor="val_acc", patience=25, mode="max", check_finite=True, check_on_train_epoch_end=False
            ),
            LearningRateMonitor(),
            ModelCheckpoint(monitor="val_acc", mode="max"),
         #   LearningRateFinder(min_lr=1e-4, num_training_steps=1000)
            ],
            logger=wandb_logger,
        )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [16]:
trainer.fit(lstm_model, train_dataloader, val_dataloader)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kpuchalskixiv. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | loss  | CrossEntropyLoss | 0     
1 | model | SimpleLSTM       | 4.8 M 
-------------------------------------------
4.8 M     Trainable params
0         Non-trainable params
4.8 M     Total params
19.136    Total estimated model params size (MB)


Epoch 25: 100%|██████████| 130/130 [00:13<00:00,  9.59it/s, v_num=keuy, train_loss=2.100, val_loss=2.060, val_acc=22.10]


In [17]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
scheduler_lr,███████████████▃▃▃▃▃▃▁▁▁▁▁
train_loss,▃▅▆▃▆▄▃▄▄▆▃▃▁▃▆▃▄▁▃▄█▂▃▄▄▃▂▃▄▃▂▃▄▄▅▇▄▄▅▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
val_acc,█████████▁████████████████
val_loss,▅▄█▂▆▂▂▂▄▁▁▅▃▂▅▂▄▃▂▄▁▁▂▂▂▂
epoch,25
scheduler_lr,0.0025
train_loss,1.88626
trainer/global_step,3379
val_acc,22.103


# Simple Transformer